In [1]:
"""
本节将使用逻辑回归预测乳腺肿瘤是良性的还是恶性的。
示例采用的数据集为威斯康星诊断乳腺癌数据集，
它通过细胞核的相关特征来预测乳腺肿瘤为良性/恶性，这是一个非常著名的二分类数据集。
该数据集包含569个样本，其中有212个恶性肿瘤样本，357个良性肿瘤样本。共有32个字段，
字段1为ID，字段2为label，其他30个字段为细胞核的相关特征，例如

    ·半径（从中心到周边点的平均距离）
    ·纹理（灰度值的标准偏差）
    ·周长·面积·光滑度（半径长度的局部变化）
    ·紧凑性（周长的二次方/面积的负一次方）
    ·凹度（轮廓的凹陷程度）
    ·凹点（轮廓中凹部的数量）
    ·对称·分形维数
"""

'\n本节将使用逻辑回归预测乳腺肿瘤是良性的还是恶性的。\n示例采用的数据集为威斯康星诊断乳腺癌数据集，\n它通过细胞核的相关特征来预测乳腺肿瘤为良性/恶性，这是一个非常著名的二分类数据集。\n该数据集包含569个样本，其中有212个恶性肿瘤样本，357个良性肿瘤样本。共有32个字段，\n字段1为ID，字段2为label，其他30个字段为细胞核的相关特征，例如\n\n    ·半径（从中心到周边点的平均距离）\n    ·纹理（灰度值的标准偏差）\n    ·周长·面积·光滑度（半径长度的局部变化）\n    ·紧凑性（周长的二次方/面积的负一次方）\n    ·凹度（轮廓的凹陷程度）\n    ·凹点（轮廓中凹部的数量）\n    ·对称·分形维数\n'

In [12]:
# 对于每张图像，分别计算以上10个特征的平均值、标准误差和最差/最大（最大的3个值的平均）值，由此生成30个特征。
# 例如，字段3表示平均半径，字段13表示半径的标准误差，字段23表示最差半径。所有特征都保留4位有效数字。
from sklearn import datasets

# 加载乳腺癌数据
cancer = datasets.load_breast_cancer()
X = cancer.data
y = cancer.target

In [21]:
from sklearn.model_selection import train_test_split
# test_size：将数据按4：1划分训练集和测试集
# random_state： 随机种子
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/5, random_state=8)

In [22]:
# 逻辑回归的API
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
# 逻辑回归
lr = LogisticRegression()
# 训练模型
lr.fit(X_train, y_train)
# 预测
y_pred = lr.predict(X_test)
# 报告()混淆矩阵
report = classification_report(y_test, y_pred, target_names=['Benign', 'Malignant'])
# 其中，列表的左边一列为分类的标签名，avg/total为各列的均值。support表示该类别样本出现的次数。
report

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


'              precision    recall  f1-score   support\n\n      Benign       0.94      0.96      0.95        46\n   Malignant       0.97      0.96      0.96        68\n\n    accuracy                           0.96       114\n   macro avg       0.95      0.96      0.95       114\nweighted avg       0.96      0.96      0.96       114\n'

In [23]:
# 利用xgboost来进行逻辑回归分析代码如下
import pandas as pd
import numpy as np
import xgboost as xgb

# 将数据转化为DMatrix格式
train_xgb = xgb.DMatrix(X_train, y_train)
# 定义模型训练参数
params = {
    "objective":'reg:logistic',
    "booster":'gblinear'
}
# 训练模型
model = xgb.train(dtrain=train_xgb, params=params)
y_pred = model.predict(xgb.DMatrix(X_test))
# XGBoost的预测结果是概率
# 下列代码将预测概率大于0.5的样本的预测分类标记为1，否则标记为0。
ypred_bsd = np.array(y_pred)
ypred_bsd = ypred_bsd > 0.5
ypred_bsd = ypred_bsd.astype(int)
ypred_bsd

array([1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1])

In [24]:
def check_predict_percent(a, b):
    right_num = 0
    for i in range(len(a)):
        if a[i] == b[i]:
            right_num += 1
    return repr(right_num/len(a))

print(check_predict_percent(ypred_bsd, y_test))

0.8947368421052632
